In [1]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)


completion = client.chat.completions.create(
    model="dummy-model",
    messages=[
      {
        "role": "user",
        "content": "You are a helpful assistant."
      },
      {
      	"role": "assistant",
      	"content": "Hello! How can I help you?"
      },
      {
      	"role": "user",
      	"content": "What is the capital of Romania?"
      }
    ],
)

In [2]:
completion.choices[0].message.content

'The capital of Romania is Bucharest.'

## Function calling

In [ ]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="dummy-model",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)

In [3]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [4]:
from openai import OpenAI
import json

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "system", "content": "You are a helpful assitant"}, {"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [5]:
completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_elOWfNfPgf56Cb0XU6I7XsF0', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]

In [6]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [7]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [8]:
completion_2.choices[0].message.content

'The current temperature in Paris is 22.4°C.'

In [9]:
messages.append({"role": "user", "content": "What's the weather like in London today?"})

In [10]:
completion_3 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [11]:
completion_3

ChatCompletion(id='chatcmpl-BtxmABqzMNwvngejb5LUEB1ExufLD', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E05zeKQtrUWryJG5R036EbTz', function=Function(arguments='{"latitude":51.5074,"longitude":-0.1278}', name='get_weather'), type='function')]))], created=1752677518, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=116, total_tokens=140, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [12]:
tool_call = completion_3.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [13]:
result

24.0

In [14]:
messages.append(completion_3.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_4 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [15]:
completion_4.choices[0].message.content

'The current temperature in Paris is 22.4°C and in London is 24.0°C.'

In [16]:
messages

[{'role': 'system', 'content': 'You are a helpful assitant'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_elOWfNfPgf56Cb0XU6I7XsF0', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_elOWfNfPgf56Cb0XU6I7XsF0',
  'content': '22.4'},
 {'role': 'user', 'content': "What's the weather like in London today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E05zeKQtrUWryJG5R036EbTz', function=Function(arguments='{"latitude":51.5074,"longitude":-0.1278}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_E05zeKQtrUWryJG5R036EbTz',
  'content': '24